In [12]:
import os
import gc
import re
import json
import pickle
import datetime
from tqdm import tqdm
from typing import Union

import numpy as np
import pandas as pd
pd.options.display.max_columns = None

from typing import Union

import seaborn
import matplotlib.pyplot as plt
plt.style.use("seaborn-darkgrid")

from scipy.stats import linregress

from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

# custom funcs
from script import WRMSSEEvaluator
from script import cache_result
from script import reduce_mem_usage
from script import load_pickle, dump_pickle
from script import get_groups

In [13]:
# TODO: submission 期間のデータも含んでしまっているので調整する必要がある
# Define variables and dataframes.
shift_days = 28
use_cols = [
    'id', 'd', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
    'sales', 'sell_price', 'dayofweek'
]

In [14]:
df=  pd.read_pickle('features/melted_and_merged_train.pkl')[use_cols]

In [15]:
dst_idx = df[['id', 'dayofweek']].to_records(index=False).tolist()

In [16]:
df = df.query('d < 1942')

In [17]:
df['sales_times_price'] = df['sales'] * df['sell_price']

In [18]:
# Convert target to binary
icols = [
    # id groups.
    ['state_id'],
#     ['store_id'],
#     ['cat_id'],
#     ['dept_id'],
#     ['state_id', 'cat_id'],
    ['state_id', 'dept_id'],
#     ['store_id', 'cat_id'],
#     ['store_id', 'dept_id'],
#     ['item_id'],
#     ['item_id', 'state_id'],
#     ['item_id', 'store_id'],
#     # datetime groups.
#     ['store_id', 'dayofweek'],
#     ['dept_id', 'dayofweek'],
#     ['item_id', 'dayofweek'],
#     ['store_id', 'dept_id', 'dayofweek'],
#     ['store_id', 'item_id', 'dayofweek']
]

targeets = ['sales', 'sales_times_price']


In [19]:
features = []
for col in icols:
    print('Encoding', col)
    col_name = '_'.join(col)
    features.extend([f'enc_{col_name}_mean', f'enc_{col_name}_std'])

    df[f'enc_{col_name}_mean'] = df.groupby(col)['sales'].transform('mean')
    df[f'enc_{col_name}_std'] = df.groupby(col)['sales'].transform('std')

Encoding ['state_id']
Encoding ['state_id', 'dept_id']


In [20]:
df.head()

,id,d,item_id,dept_id,cat_id,store_id,state_id,sales,sell_price,dayofweek,sales_times_price,enc_state_id_mean,enc_state_id_std,enc_state_id_dept_id_mean,enc_state_id_dept_id_std
0,HOBBIES_1_001_CA_1_evaluation,819,1437,3,1,0,0,0,NaN,4,NaN,1.3376,4.015207,0.98047,2.893269
1,HOBBIES_1_002_CA_1_evaluation,819,1438,3,1,0,0,0,3.970703,4,0.000000,1.3376,4.015207,0.98047,2.893269
2,HOBBIES_1_003_CA_1_evaluation,819,1439,3,1,0,0,0,NaN,4,NaN,1.3376,4.015207,0.98047,2.893269
3,HOBBIES_1_004_CA_1_evaluation,819,1440,3,1,0,0,1,4.640625,4,4.640625,1.3376,4.015207,0.98047,2.893269
4,HOBBIES_1_005_CA_1_evaluation,819,1441,3,1,0,0,1,3.080078,4,3.080078,1.3376,4.015207,0.98047,2.893269


In [21]:
df = df.set_index(['id', 'dayofweek'])

In [25]:
df.loc[dst_idx[:10], features].head()

enc_state_id_mean  enc_state_id_std  \
id                            dayofweek                                        
HOBBIES_1_001_CA_1_evaluation 4                     1.3376          4.015207   
                              4                     1.3376          4.015207   
                              4                     1.3376          4.015207   
                              4                     1.3376          4.015207   
                              4                     1.3376          4.015207   

                                         enc_state_id_dept_id_mean  \
id                            dayofweek                              
HOBBIES_1_001_CA_1_evaluation 4                            0.98047   
                              4                            0.98047   
                              4                            0.98047   
                              4                            0.98047   
                              4                            0.98047   

                                         enc_state_id_dept_id_std  
id                            dayofweek                            
HOBBIES_1_001_CA_1_evaluation 4                          2.893269  
                              4                          2.893269  
                              4                          2.893269  
                              4                          2.893269  
                              4                          2.893269